# 데이터 분석 및 모델링

본 노트북은 요기요 리뷰 데이터를 분석하여 고객 요구사항을 식별하고 분류 모델을 구축하는 과정을 담고 있습니다.

## 0. 필수 라이브러리 설치

### Subtask:
필요한 라이브러리를 설치합니다.

In [ ]:
!pip install gensim rake-nltk scikit-learn tensorflow imblearn wordcloud # 필요한 라이브러리 모두 포함

## 1. 라이브러리 불러오기

### Subtask:
분석에 필요한 모든 라이브러리를 불러옵니다.

In [ ]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, accuracy_score
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
import nltk
from rake_nltk import Rake
from imblearn.over_sampling import SMOTE
from wordcloud import WordCloud
from collections import Counter
import matplotlib.font_manager as fm # 폰트 매니저 임포트
import os # 폰트 경로 확인 등 필요

## 2. 데이터 로드

### Subtask:
원본 데이터인 'yogiyo_reviews_30000.csv' 파일을 로드합니다.

**Reasoning**:
분석을 위해 원본 데이터 파일을 pandas DataFrame으로 로드합니다.

In [ ]:
df = pd.read_csv("yogiyo_reviews_30000.csv")
display(df.head())

## 3. 사용자 이름 비식별화

### Subtask:
사용자 이름 열의 개인 정보를 보호하기 위해 블리딩 처리를 수행합니다.

**Reasoning**:
Define a function to mask user names and apply it to the 'userName' column to create a new 'masked_userName' column. Then, drop the original 'userName' column and display the head of the dataframe to verify the changes.

In [ ]:
def mask_username(username):
    if isinstance(username, str) and len(username) > 1:
        return username[0] + '*' * (len(username) - 1)
    return username

df['masked_userName'] = df['userName'].apply(mask_username)
df = df.drop('userName', axis=1)
display(df.head())

## 4. 초기 데이터 전처리

### Subtask:
리뷰 내용을 토큰화하고 기본적인 전처리를 수행합니다.

**Reasoning**:
Define the tokenize function to remove non-Korean characters and split the text into words. Then, apply this function to the 'content' column and join the tokens back into strings for further processing.

In [ ]:
def tokenize(text):
    text = re.sub(r"[^\uAC00-\uD7A3\s]", "", str(text))
    return text.split()

tokenized_texts = df['content'].apply(tokenize).tolist()
texts_joined = [' '.join(tokens) for tokens in tokenized_texts]

## 5. 별점 기반 요구사항 재 라벨링

### Subtask:
별점을 기준으로 요구사항 라벨을 재설정합니다. (4점 이하 리뷰는 요구사항이 있다고 가정)

**Reasoning**:
Based on the user's revised strategy, create a new boolean column 'requirement' where True indicates a score less than or equal to 4, and False otherwise.

In [ ]:
df['requirement'] = df['score'] <= 4
display(df.head())
display(df['requirement'].value_counts())

## 6. 목표 변수 설정

### Subtask:
감성 라벨과 별점 기반으로 재 라벨링된 요구사항 라벨을 모델의 목표 변수로 설정합니다.

**Reasoning**:
Define the target variables `y_sentiment` and `y_requirement`. Sentiment is inferred from the score (<= 3 is negative, > 3 is positive). The requirement label is based on the newly created 'requirement' column (True/False converted to 1/0).

In [ ]:
# 감성 라벨 (별점 기준 재정의: 3점 이하 negative, 4점 이상 positive)
df['sentiment'] = df['score'].apply(lambda x: 0 if x <= 3 else 1)
y_sentiment = df['sentiment'].values

# 요구사항 라벨 (False/True → 0/1)
y_requirement = df['requirement'].astype(int).values

print("✅ 감성 라벨 분포:")
display(pd.Series(y_sentiment).value_counts())
print("\n✅ 요구사항 라벨 분포:")
display(pd.Series(y_requirement).value_counts())

## 7. Word2Vec 임베딩 및 임베딩 행렬 생성

### Subtask:
전처리된 텍스트 데이터를 사용하여 Word2Vec 모델을 학습시키고 임베딩 행렬을 생성합니다.

**Reasoning**:
Train a Word2Vec model on the `tokenized_texts` and create an embedding matrix for the vocabulary.

In [ ]:
embedding_dim = 100
w2v_model = Word2Vec(sentences=tokenized_texts, vector_size=embedding_dim, window=5, min_count=2, workers=2)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts_joined)
sequences = tokenizer.texts_to_sequences(texts_joined)
word_index = tokenizer.word_index
vocab_size = len(word_index) + 1
max_len = 30
X = pad_sequences(sequences, maxlen=max_len)

embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in word_index.items():
    if word in w2v_model.wv:
        embedding_matrix[i] = w2v_model.wv[word]

## 8. 데이터 분할

### Subtask:
학습 및 평가를 위해 데이터를 분할합니다.

**Reasoning**:
Split the data into training and testing sets for both sentiment and requirement labels using `train_test_split`. The test size is set to 20% and a random state is used for reproducibility.

In [ ]:
X_train, X_test, y_sentiment_train, y_sentiment_test, y_requirement_train, y_requirement_test = train_test_split(
    X, y_sentiment, y_requirement, test_size=0.2, random_state=42
)

## 9. 데이터 불균형 해소 (SMOTE 적용)

### Subtask:
요구사항 분류의 데이터 불균형을 해소하기 위해 SMOTE를 적용합니다.

**Reasoning**:
Apply SMOTE to the training data (`X_train`, `y_requirement_train`) to oversample the minority class ('requirement 있음'). This will help address the data imbalance issue and potentially improve the model's performance on the minority class. Note that SMOTE should only be applied to the training data, not the test data.

In [ ]:
# SMOTE 객체 초기화
smote = SMOTE(random_state=42)

# 요구사항 훈련 데이터에 SMOTE 적용
# Note: 이 과정에서 X_train과 y_requirement_train의 샘플 수가 증가합니다.
# y_sentiment_train은 그대로이므로 다음 모델 학습 단계에서 샘플 수 불일치 오류가 발생할 수 있습니다.
X_train_res, y_requirement_train_res = smote.fit_resample(X_train, y_requirement_train)

print("✅ SMOTE 적용 전 요구사항 훈련 데이터 라벨 분포:")
display(pd.Series(y_requirement_train).value_counts())
print("\n✅ SMOTE 적용 후 요구사항 훈련 데이터 라벨 분포:")
display(pd.Series(y_requirement_train_res).value_counts())

# Note: y_sentiment_train_res는 SMOTE 적용 결과에 포함되지 않으므로 이 줄은 제거합니다.
# print("\n✅ SMOTE 적용 후 감성 훈련 데이터 라벨 분포:") # 감성 라벨 분포도 확인
# display(pd.Series(y_sentiment_train_res).value_counts())

## 10. 모델 정의 및 학습 (SMOTE 적용 데이터 활용)

### Subtask:
재 라벨링된 요구사항 라벨과 SMOTE 적용 데이터를 사용하여 모델을 정의하고 학습합니다. (LSTM 128, Dropout 0.3, Early Stopping 적용)

**Reasoning**:
Define a neural network model using the Keras functional API with an Embedding layer initialized with the Word2Vec embedding matrix, an LSTM layer, and Dropout. The model has two output layers for sentiment and requirement classification. Compile the model and train it using the training data, including the SMOTE-resampled requirement data, with Early Stopping. Address the sample size mismatch for the sentiment output by aligning it with the resampled data size.

In [ ]:
# EarlyStopping 콜백 설정
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True,
    verbose=1
)

# 모델 구성 (LSTM 128, Dropout 0.3)
input_ = Input(shape=(max_len,))
x = Embedding(input_dim=vocab_size, output_dim=embedding_dim, weights=[embedding_matrix], input_length=max_len, trainable=True)(input_)
x = LSTM(128, return_sequences=False)(x)
x = Dropout(0.3)(x)

out_sentiment = Dense(1, activation='sigmoid', name='sentiment_output')(x)
out_requirement = Dense(1, activation='sigmoid', name='requirement_output')(x)

model = Model(inputs=input_, outputs=[out_sentiment, out_requirement])
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics={'sentiment_output': 'accuracy', 'requirement_output': 'accuracy'}
)
model.summary()

# 모델 학습 (SMOTE 적용된 요구사항 훈련 데이터 사용)
# Note: SMOTE만 요구사항 데이터에 적용되었으므로, sentiment는 원본 훈련 데이터 사용
# ValueError 해결을 위해 y_sentiment_train의 샘플 수를 X_train_res에 맞춥니다 (데이터 복제 - 이상적이지 않을 수 있음)
# 이 부분은 다중 출력 모델과 SMOTE 적용 시 발생할 수 있는 샘플 불일치 문제에 대한 임시 해결책입니다.
num_samples_res = X_train_res.shape[0]
# Replicate y_sentiment_train to match the number of samples in X_train_res
y_sentiment_train_aligned = np.tile(y_sentiment_train, (num_samples_res // y_sentiment_train.shape[0]) + 1)[:num_samples_res]


history = model.fit(
    X_train_res, # SMOTE 적용된 데이터 사용
    [y_sentiment_train_aligned, y_requirement_train_res], # sentiment는 샘플 수 맞춤, requirement는 SMOTE 적용 데이터 사용
    epochs=15,
    batch_size=64,
    validation_split=0.1, # 검증 데이터는 SMOTE 적용 전 데이터의 일부에서 분할
    callbacks=[early_stop],
    verbose=2
)

## 11. 모델 평가

### Subtask:
학습된 모델의 성능을 평가하고 confusion matrix를 시각화합니다.

**Reasoning**:
Evaluate the trained model using the test data and print classification reports for both sentiment and requirement. Visualize the confusion matrices for both tasks.

In [ ]:
pred_sent, pred_req = model.predict(X_test)
pred_sent = (pred_sent > 0.5).astype(int).flatten()
pred_req = (pred_req > 0.5).astype(int).flatten()

print("✅ 감성 분류 평가")
print(classification_report(y_sentiment_test, pred_sent))

print("\n✅ 요구사항 분류 평가")
print(classification_report(y_requirement_test, pred_req))

cm1 = confusion_matrix(y_sentiment_test, pred_sent)
disp1 = ConfusionMatrixDisplay(confusion_matrix=cm1)
disp1.plot(cmap='Purples')
plt.title("Sentiment Confusion Matrix")
plt.show()

cm2 = confusion_matrix(y_requirement_test, pred_req)
disp2 = ConfusionMatrixDisplay(confusion_matrix=cm2)
disp2.plot(cmap='Greens')
plt.title("Requirement Confusion Matrix")
plt.show()

## 12. 모델 성능 요약표

### Subtask:
학습된 모델의 감성 및 요구사항 분류 성능 지표를 표로 요약합니다.

**Reasoning**:
Calculate or extract the key performance metrics (Precision, Recall, F1-Score, Accuracy) for both sentiment and requirement classification from the test set predictions. Organize these metrics into a pandas DataFrame for a clear and concise summary table.

In [ ]:
# 감성 분류 성능 지표 계산
sentiment_accuracy = accuracy_score(y_sentiment_test, pred_sent)
sentiment_precision = precision_score(y_sentiment_test, pred_sent, average='weighted')
sentiment_recall = recall_score(y_sentiment_test, pred_sent, average='weighted')
sentiment_f1 = f1_score(y_sentiment_test, pred_sent, average='weighted')

# 요구사항 분류 성능 지표 계산
requirement_accuracy = accuracy_score(y_requirement_test, pred_req)
requirement_precision = precision_score(y_requirement_test, pred_req, average='weighted')
requirement_recall = recall_score(y_requirement_test, pred_req, average='weighted')
requirement_f1 = f1_score(y_requirement_test, pred_req, average='weighted')

# 성능 지표를 DataFrame으로 정리
performance_summary = pd.DataFrame({
    'Task': ['Sentiment Classification', 'Requirement Classification'],
    'Accuracy': [sentiment_accuracy, requirement_accuracy],
    'Precision (Weighted)': [sentiment_precision, requirement_precision],
    'Recall (Weighted)': [requirement_recall, requirement_recall], # 수정: requirement_recall로 통일
    'F1-Score (Weighted)': [sentiment_f1, requirement_f1]
})

display(performance_summary)

## 13. 모델 학습 과정 시각화

### Subtask:
모델 학습 중 손실(loss)과 정확도(accuracy) 변화를 시각화합니다.

**Reasoning**:
Extract the training and validation loss and accuracy from the `history` object returned by `model.fit`. Plot these values over epochs using Matplotlib to visualize the model's learning progress.

In [ ]:
# 손실 그래프
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

# 정확도 그래프 (감성 분류)
plt.figure(figsize=(12, 6))
plt.plot(history.history['sentiment_output_accuracy'], label='Train Sentiment Accuracy')
plt.plot(history.history['val_sentiment_output_accuracy'], label='Validation Sentiment Accuracy')
plt.title('Sentiment Classification Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

# 정확도 그래프 (요구사항 분류)
plt.figure(figsize=(12, 6))
plt.plot(history.history['requirement_output_accuracy'], label='Train Requirement Accuracy')
plt.plot(history.history['val_requirement_output_accuracy'], label='Validation Requirement Accuracy')
plt.title('Requirement Classification Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)
plt.show()

## 14. 요구사항 리뷰 키워드 분석 (RAKE)

### Subtask:
요구사항이 있다고 판단된 리뷰들을 대상으로 RAKE를 활용하여 핵심 키워드를 추출합니다.

**Reasoning**:
Use RAKE to extract top keywords from reviews identified as containing requirements based on the re-labeling strategy. Define custom Korean stopwords and apply the RAKE algorithm to the content of the relevant reviews. Display a few examples and save the results to an Excel file.

In [ ]:
# 불용어 오류 방지를 위한 리소스 다운로드
try:
    nltk.data.find('corpora/stopwords')
except nltk.downloader.DownloadError:
    nltk.download('stopwords')
try:
    nltk.data.find('tokenizers/punkt_tab')
except nltk.downloader.DownloadError:
    nltk.download('punkt_tab')


# 1. 요구사항 있는 리뷰만 추출
df_req = df[df['requirement'] == True].copy()
print(f"요구사항 포함 리뷰 수: {len(df_req)}")

# 2. 한국어 불용어 수동 지정 (RAKE는 영어 기준 → 한국어 맞춤 대응)
korean_stopwords = ['이', '그', '저', '더', '좀', '정말', '진짜', '그리고', '때문에', '것', '수', '있다', '없다', '에서', '입니다']

# 3. RAKE 객체 초기화
rake = Rake(stopwords=korean_stopwords)

# 4. 키워드 추출 함수 정의
def extract_keywords_rake(text, top=3):
    rake.extract_keywords_from_text(str(text))
    return rake.get_ranked_phrases()[:top]

# 5. 키워드 추출 적용
df_req['keywords'] = df_req['content'].apply(lambda x: extract_keywords_rake(x))

# 6. 일부 결과 출력
for i in range(min(10, len(df_req))):
    print(f"리뷰: {df_req.iloc[i]['content']}")
    print(f"핵심 키워드: {df_req.iloc[i]['keywords']}")
    print('-'*60)

# 7. 결과를 엑셀로 저장
df_req[['content', 'keywords']].to_excel("요구사항_리뷰_핵심키워드.xlsx", index=False)
print("✅ 결과 저장 완료: '요구사항_리뷰_핵심키워드.xlsx'")

## 15. 요구사항 핵심 키워드 워드 클라우드 시각화 (부정 리뷰 기반)

### Subtask:
부정적인 요구사항 리뷰에서 추출된 핵심 키워드를 워드 클라우드로 시각화합니다.

**Reasoning**:
Generate a word cloud from the extracted keywords specifically from the negative requirement reviews (`df_negative_req`) to visually represent the most frequent and prominent terms in those reviews. This visualization can highlight key areas for improvement for the company by focusing on negative feedback.

In [ ]:
# 1. 부정적인 감성이면서 요구사항이 있다고 판단된 리뷰만 추출 (이전 셀에서 생성된 df_negative_req 사용)
# df_negative_req = df[(df['sentiment'] == 0) & (df['requirement'] == True)].copy() # 이미 위에서 생성됨

# 2. 추출된 키워드 리스트를 하나의 문자열로 합치기 (부정적인 요구사항 리뷰만 사용)
all_keywords_negative_req = []
# RAKE 객체를 새로 초기화하거나, df_negative_req에 대해 RAKE를 다시 적용해야 합니다.
# 여기서는 df_negative_req의 'content' 열에 대해 RAKE를 다시 적용하는 예시 코드를 포함합니다.

# RAKE 객체 초기화 (불용어 포함)
korean_stopwords = ['이', '그', '저', '더', '좀', '정말', '진짜', '그리고', '때문에', '것', '수', '있다', '없다', '에서', '입니다']
rake_neg = Rake(stopwords=korean_stopwords) # 새로운 RAKE 객체 사용

# 키워드 추출 함수 정의
def extract_keywords_rake_neg(text, top=5): # 상위 키워드 개수를 5개로 늘려보겠습니다.
    rake_neg.extract_keywords_from_text(str(text))
    return rake_neg.get_ranked_phrases()[:top]

# 키워드 추출 적용
df_negative_req['keywords_neg'] = df_negative_req['content'].apply(lambda x: extract_keywords_rake_neg(x))


for keyword_list in df_negative_req['keywords_neg']:
    all_keywords_negative_req.extend(keyword_list)


# 3. 키워드 빈도 계산
keyword_counts_negative_req = Counter(all_keywords_negative_req)

# 4. 한국어 폰트 설정 (Colab 환경에 맞게 수정)
# 런타임 다시 시작 후에는 아래 설정만 필요합니다.
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지

# 5. 워드 클라우드 생성
wordcloud_neg = WordCloud(
    font_path='/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # 설치된 나눔 폰트 경로
    width=800,
    height=400,
    background_color='white'
).generate_from_frequencies(keyword_counts_negative_req)

# 6. 워드 클라우드 시각화
plt.figure(figsize=(15, 10))
plt.imshow(wordcloud_neg, interpolation='bilinear')
plt.axis('off')
plt.title('부정적인 요구사항 리뷰 핵심 키워드 워드 클라우드')
plt.show()

## 16. 요구사항 핵심 키워드 빈도 막대 그래프 시각화 (부정 리뷰 기반)

### Subtask:
부정적인 요구사항 리뷰에서 추출된 핵심 키워드의 빈도를 막대 그래프로 시각화합니다.

**Reasoning**:
Calculate the frequency of the extracted keywords from the negative requirement reviews (`df_negative_req`). Select the top N most frequent keywords and create a bar chart using Matplotlib to visualize their distribution. This provides a quantitative view of the most pressing issues mentioned in negative requirement reviews.

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter
import pandas as pd
import matplotlib.font_manager as fm # 폰트 매니저 임포트

# 1. 부정적인 요구사항 리뷰에서 추출된 키워드 리스트 가져오기
# 이전 셀에서 df_negative_req와 keyword_counts_negative_req가 생성되었다고 가정합니다.

# 2. 키워드 빈도 계산 (이전 셀에서 계산된 keyword_counts_negative_req 사용)
# keyword_counts_negative_req = Counter(all_keywords_negative_req) # 이미 계산됨

# 3. 상위 N개 키워드 선택
top_n = 30 # 원하는 상위 키워드 개수
top_keywords_neg = keyword_counts_negative_req.most_common(top_n)
top_keywords_dict_neg = dict(top_keywords_neg)

# 4. 시각화를 위한 데이터 준비
keywords_neg = list(top_keywords_dict_neg.keys())
counts_neg = list(top_keywords_dict_neg.values())

# 5. 막대 그래프 시각화
plt.figure(figsize=(12, 8))

# 한국어 폰트 설정
plt.rc('font', family='NanumGothic')
plt.rcParams['axes.unicode_minus'] = False # 마이너스 부호 깨짐 방지

plt.barh(keywords_neg, counts_neg, color='skyblue')
plt.xlabel('빈도수')
plt.ylabel('핵심 키워드')
plt.title(f'부정적인 요구사항 리뷰 상위 {top_n}개 핵심 키워드 빈도')
plt.gca().invert_yaxis() # 빈도수 높은 순으로 위에서부터 표시
plt.show()